In [ ]:
import os, time, re
import pandas as pd

In [ ]:
def split_last_3_cols(text):
    # reverse
    text = text[::-1]
    text = text.split(',', 2)
    text.reverse()
    text = [x[::-1] for x in text]
    text[-1] = text[-1].replace('\n', '')
    return text # is a list

def parse_before_notes(text):
    state = 'MA'
    start = text.find(',MA,')
    if start < 0:
        start = text.find('{')
        data_1 = text[:start-1].split(',', 12)[:-1]
        data_2 = text[start:].split(',', 13)
        data = data_1 + data_2[:-1]
        return data, data_2[-1]
    
    left = text[:start][::-1].split(',', 1)
    city = left[0][::-1]
    address = left[-1][::-1].split(',', 2)
    if len(address) < 3:
        address += [''] * (3 - len(address))
    
    end = start + 4
    right = text[end:]
    right = right.split(',', 1)
    
    if len(right[0]) < 4 or not right[0].isdigit():
        zip = ''
    else:
#         if len(zip) == 4:
#             zip
        zip = right[0]
    
    
    data = address + [city, state, zip]
    
    right = right[-1].split(',', 24)
    
    data += right[:-1]
    
    return data, right[-1]

In [ ]:
pd.read_excel('./data/CPCS TRIS Source Data/Priority2-RunsheetData-Full.xlsx')

In [ ]:
with open('./data/CPCS TRIS Source Data/Priority2-RunsheetData-Full.csv', mode='r', encoding='cp1252') as f:
    lines = f.readlines()
len(lines)    

# f = open('./data/CPCS TRIS Source Data/Priority2-RunsheetData-Full.csv', mode='r', encoding='cp1252')

In [ ]:
# lines[0].split(',')
headers = ['LOG_ID',
 'CASE_ID',
 'CASE_NAME',
 'CASE_OPENED',
 'CLIENT_ID',
 'CLIENT NAME',
 'address_line1',
 'address_line2',
 'address_line3',
 'city',
 'state',
 'zip',
 'RACEID',
 'RACE',
 'ETHNICITYID',
 'ETHNICITY',
 'GENDER',
 'DOB',
 'ATTORNEYASSIGNED_1',
 'ATTORNEYASSIGNED_2',

 'COURTID',
 'COURT',
 'ABBR',
 'RECORDEDBY GUID',
 'RECORDEDBY',

 'ASSIGNEDTO_1',
 'ASSIGNEDTO_2',
 'ASSIGNEDTO_3',

 'DATE_RECORDED',
 'ACTIVITY_DATE',
 'ACTIVITY_TYPE',
 'ACTIONTYPE',
 'ACTIVITY_SUBTYPE',
 'ACTIONSUBTYPE',
 'OUTCOME',
 'OUTCOME NAME',
 'NOTES',
 'OFFICE_ID',
 'OFFICE NAME']
# 70699

In [ ]:
# df = pd.DataFrame(columns=lines[0].split(','))
# headers = lines[0].split(',')
pattern = re.compile('[\d]{4,}-[\d]{2,}-[\d]{2,} [\d]{2,}:[\d]{2,}:[\d]{2,}')
err = []
data = []

i = 1
maxline = 1032247 # len(lines)=1032247

start = time.time()
while i < maxline - 1:
    dict_data = {}
    row_end = False
    line = lines[i]
    # cut before address

    line = line.split(',', 6)
    match = None
    for t in range(3, 6+1):
        match = re.search(pattern, line[t][:19])
        if match: break
            
    if match:
        row = line[:2] + [','.join(line[2:t])] + line[t:-1]
        line = ','.join(line).split(',', 6 + t - 3)
#         print(row)
    else:
        row = line[:-1]
        
    before_notes, last = parse_before_notes(line[-1])
    row += before_notes
    
    # next, parse notes
    # log id has at least 4 digits
    notes = last
    j = i + 1
    while j < maxline and not row_end:
        next_line = lines[j]
        tmp = next_line.split(',', 33)
        if len(tmp) == 34 \
                and len(next_line) >= 4 \
                and len(tmp[0]) >= 4 \
                and tmp[0].isdigit() \
                and tmp[1].isdigit():
            row_end = True
            i = j
            break
        
        # notes not end yet
        else:
            notes += next_line
            j += 1

    notes_with_3_col = split_last_3_cols(notes)
    row += notes_with_3_col
    
    if '.' not in row[6]:
        err.append(row)
        continue
    try:
        dict_data = {headers[k]: row[k].strip() for k in range(len(headers))}
    except Exception as e:
        err.append(row)
#         print(e)
#         break
    else:
        data.append(dict_data)
        if i % 10000 == 0:
            print(i, end='\t')

print(f'cost time: {time.time() - start}\n')

d= {}
for i, entry in enumerate(data):
    # add a dictionary entry to the final dictionary
    d[i] = entry

print(f'err nums: {len(err)}\n')
df = pd.DataFrame.from_dict(d, orient='index', columns=headers)
df

In [ ]:
start = time.time()
df.to_excel('./data/CPCS TRIS Source Data/Priority2-RunsheetData-delete14187rows-clean.xlsx', encoding='utf8', index=None)
print(f'cost time: {time.time() - start}')

In [ ]:
# df[(0 < df['CASE_OPENED'].str.len()) & (df['CASE_OPENED'].str.len() < 19)]

# fix err, no need
# for x in err:
#     for i in range(5):
#         del x[18]
#         x.insert(18, '')
# df.loc[0, 'address_line1']

In [ ]:
# tmp = pd.DataFrame(err, columns=headers)
# tmp = df.append(tmp, ignore_index=True)

In [ ]:
for i in range(len(err)):
    if err[i][0] == '70699':
        print(i, len(err[i]), err[i])

In [ ]:
match

In [ ]:
i, len(row), len(data)

In [ ]:
def a(s):
    s = s.split(',')
    return s

s = '1,2,3,4,5'
s.split(',', 0)
# a(s), s
# len(row), type(row)

In [ ]:
a = [1,22,3,4,5]
a[4:-1]

In [ ]:
len('2009-11-12 00:00:00')

In [ ]:
pattern = re.compile(r'[\d]{4,}-[\d]{2,}-[\d]{2,} [\d]{2,}:[\d]{2,}:[\d]{2,}')
index = re.search(pattern, '2009-11-12 00:00:00')
if index:
    print(index)